In [15]:
import pandas as pd
import os

# Define the file path
file_path = '/home/username/Documents/MFEC/Tax Advisor/dirtydata.csv'
cleaned_file_path = '/home/username/Documents/MFEC/Tax Advisor/cleaneddata.csv'

# Check if the file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Remove empty columns
    df.dropna(axis=1, how='all', inplace=True)

    # Remove columns with names that are just spaces or not found
    df.columns = [col.strip() for col in df.columns]  # Remove leading/trailing spaces in column names
    df = df.loc[:, df.columns != '']

    # Save the cleaned dataframe to a new CSV file
    df.to_csv(cleaned_file_path, index=False)

    print("Data cleaning completed. Cleaned data saved to:", cleaned_file_path)

File not found: /home/username/Documents/MFEC/Tax Advisor/dirtydata.csv


# Langchain OpenAI Api Embedded using Query for Faster Context, however it'll be quite inaccurate...
**Recommended to Embed in Documents for higher accuracy**

In [17]:
import os
import pandas as pd
import json
from langchain_openai import AzureOpenAIEmbeddings
from tqdm import tqdm
import json
from tqdm import tqdm
# Azure OpenAI Configuration
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://insideout.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "b6c0397877a9420aa7b3c2bad2e622f7"
os.environ["OPENAI_EMBEDDINGS_DEPLOYMENT"] = "text-embedding-3-small"

# Load the CSV file
df = pd.read_csv('/home/lord/MFEC/Yong-MFEC-Intern/TaxAdvisor/finally1.csv')

# Extract the first 10 rows of the Inquiry and Conclusion columns

inquiries = df['Inquiry']
conclusions = df['Conclusion']

# Initialize the AzureOpenAIEmbeddings class
embeddings = AzureOpenAIEmbeddings(deployment=os.environ["OPENAI_EMBEDDINGS_DEPLOYMENT"])

def embed_text(text):
    return embeddings.embed_query(text)

In [18]:
import json
import numpy as np

embedded_inquiries = [embeddings.embed_query(inquiry.replace("\n", "")) for inquiry in tqdm(inquiries)]

# Create a dictionary to store the embeddings and their corresponding conclusions
embedData = {}
for idx, embedding in enumerate(embedded_inquiries):
    embedData[idx] = {
        'embedding': embedding,
        'conclusion': conclusions[idx]
    }

# Save the embedded data to a JSON file
with open("./embedtaxdata.jsonl", "w", encoding="UTF-8") as file:
    json.dump(embedData, file, ensure_ascii=False)

  0%|          | 5/1644 [00:02<11:31,  2.37it/s]

KeyboardInterrupt: 

In [19]:
import numpy as np
import json
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import plotly.express as px

# Load the embeddings
with open("/home/lord/MFEC/Yong-MFEC-Intern/TaxAdvisor/embedtaxdata.jsonl", "r", encoding="UTF-8") as file:
    data = json.load(file)
    vectorData = np.array([v['embedding'] for v in data.values()])

# Perform dimensionality reduction with Truncated SVD
tsvd = TruncatedSVD(n_components=2)
vectorData_tsvd = tsvd.fit_transform(vectorData)

# Plot Truncated SVD results
fig_tsvd = px.scatter(x=vectorData_tsvd[:, 0], y=vectorData_tsvd[:, 1], title="Truncated SVD")
fig_tsvd.show()

# Initialize t-SNE and fit the data with perplexity set to 5
tsne = TSNE(n_components=2, perplexity=5)
vectorData_tsne = tsne.fit_transform(vectorData_tsvd)

# Plot t-SNE results
fig_tsne = px.scatter(x=vectorData_tsne[:, 0], y=vectorData_tsne[:, 1], title="t-SNE")
fig_tsne.show()

#Cosine Search

In [20]:
from pymilvus import MilvusClient
import json
import numpy as np

# Open the embedded file and turn them into values
file = open("/home/lord/MFEC/Yong-MFEC-Intern/TaxAdvisor/embedtaxdata.jsonl", "r", encoding="UTF-8")
data = json.load(file)
vectorData = np.array([v['embedding'] for v in data.values()])
vectorKeys = list(data.keys())
file.close()

# Create/get the database
mil = MilvusClient("taxinfos.db")
collectionName = "taxs"

# If the collection already exists, drop it
if mil.has_collection(collectionName):
    mil.drop_collection(collectionName)

# Create the new collection
# Dimensions = the dimensions in one embedding for a particular sample; for azure embedding 3 small dimensions = 1536
mil.create_collection(collection_name=collectionName, dimension=vectorData.shape[1])

# Milvus database entry format: #{id:id, vector: vector, text:text} - for the ID number just use the of the value in the normal list 
# Reformat out stuff to line up with that
# Not putting in the text - id is enough to identify
milDict = [{"id": i, "vector":vectorData[i]} for i in range(len(vectorData))]
mil.insert(collectionName, milDict)



{'insert_count': 1644,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

In [21]:
from pymilvus import MilvusClient

mil = MilvusClient("taxinfos.db")
collectionName = "taxs"

In [24]:
import json

query = input("Enter query: ")

# Embed_text function was declared way above, might need to rerun that cell in order to have it be effective here
milResults = mil.search(collection_name=collectionName, data=[embed_text(query)], limit=10)

data = {str(key): value for key, value in data.items()}

with open("/home/lord/MFEC/Yong-MFEC-Intern/TaxAdvisor/embedtaxdata.jsonl", "r", encoding="UTF-8") as file:
    data = json.load(file)

# Get the results in the database for the ones given by Milvus
results = [dataDict["id"] for dataDict in milResults[0]]

# Print it out for the user
print(f"Results for query: {query}")
for result in results:
    inquiry_text = data[str(result)]['embedding']
    conclusion_text = data[str(result)]['conclusion']
    print(f"Conclusion: {conclusion_text}")

Results for query: ภาษีบ้าน
Conclusion: 175.17บาท"
Conclusion: กรณีนายก.ได้ขายอสังหาริมทรัพย์ที่ใช้เป็นที่อยู่อาศัยแห่งเดิมโดยมีชื่ออยู่ในทะเบียนบ้านตามกฎหมายว่าด้วยการทะเบียนราษฎรเป็นเวลาไม่น้อยกว่าหนึ่งปีนับแต่วันที่ได้รับมาซึ่งกรรมสิทธิ์หรือได้สิทธิครอบครองในอสังหาริมทรัพย์ดังกล่าวและได้ซื้ออสังหาริมทรัพย์ที่ใช้เป็นที่อยู่อาศัยแห่งใหม่เพื่อใช้เป็นที่อยู่อาศัยของตนภายในกำหนดเวลาหนึ่งปีนับแต่วันที่ขายอสังหาริมทรัพย์ที่ใช้เป็นที่อยู่อาศัยแห่งเดิมโดยย้ายทะเบียนบ้านเดิมและเข้ามาอยู่ในทะเบียนบ้านหลังใหม่นายก.จึงได้รับยกเว้นภาษีเงินได้และอากรแสตมป์ดังต่อไปนี้1.กรณีภาษีเงินได้บุคคลธรรมดานายก.ได้รับยกเว้นภาษีเงินได้สำหรับเงินได้จากการขายอสังหาริมทรัพย์ที่ใช้เป็นที่อยู่อาศัยอันเป็นแหล่งสำคัญแห่งเดิมแต่ไม่เกินจำนวนมูลค่าของอสังหาริมทรัพย์แห่งใหม่เพื่อใช้เป็นที่อยู่อาศัยของตนตามข้อ2(62)แห่งกฎกระทรวงฉบับที่126(พ.ศ.2509)ออกตามความในประมวลรัษฎากรว่าด้วยการยกเว้นรัษฎากรประกอบกับประกาศอธิบดีกรมสรรพากรเกี่ยวกับภาษีเงินได้(ฉบับที่125)เรื่องกำหนดหลักเกณฑ์วิธีการและเงื่อนไขเพื่อการยกเว้นภาษีเงินได้สำหรั

In [27]:
import os
import numpy as np
from pymilvus import MilvusClient
from openai import AzureOpenAI
import json


# Initialize OpenAI client with Azure-specific settings
client = AzureOpenAI(
    api_version="2024-02-01",
    azure_endpoint="https://insideout.openai.azure.com/",
    api_key="b6c0397877a9420aa7b3c2bad2e622f7"
)

# Generate response using the OpenAI Chat API
similar_queries_and_answers = {
    result: [data.get(result, {}).get('conclusion')] for result in results
}
response = client.chat.completions.create(
    model="gpt4o",  # Adjust the model name if necessary
    messages = [
    {
        "role": "system",
        "content": "You are a lawyer that accepts a query from the user along with some data which contains answers for similar queries and answers to those queries to the one that the user inputted. The queries will be in an array with each entry being in the form of query:[answer, citation]. You will use that data given and make an educated guess on the correct answer to the user's query. Don't tell the user that you are making an educated guess. Sound sure of your answer."
    },
    {
        "role": "user",
        "content": f"User query: {query}\nSimilar queries and answers: {similar_queries_and_answers}\nProvide answer to user query, along with some citations for information that you believe is relevant to the user's query and support your answer with evidence."
    }
])


# Print the response from the OpenAI APIdd
print(response.choices[0].message.content.strip())


ภาษีบ้านในประเทศไทยต่อสำนักงานสรรพากรจะกำหนดขึ้นตามมูลค่าของที่ดินและสิ่งปลูกสร้าง โดยมีข้อกำหนดและอัตราภาษีที่ต้องปฏิบัติตามตามกฎหมายที่เกี่ยวข้อง ซึ่งแบ่งออกตามประเภทของการใช้ประโยชน์ที่ดิน เช่น การใช้เป็นที่อยู่อาศัย การใช้ในเชิงพาณิชย์ หรือการใช้ในเกษตรกรรม

สำหรับข้อมูลเพิ่มเติมและรายละเอียดล่าสุดของอัตราภาษี เราขอแนะนำให้คุณตรวจสอบเว็บไซต์ของสำนักงานสรรพากรที่ http://www.rd.go.th หรือติดต่อสายด่วนสรรพากรที่ 1161.

อ้างอิง:
- สำนักงานสรรพากร, “ภาษีเกี่ยวกับที่ดินและสิ่งปลูกสร้าง,” http://www.rd.go.th
